In [3]:
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D
from keras.layers import Flatten, Dropout, Dense, Input, concatenate
from keras import Model

IMSIZE = 224
input_layer = Input([IMSIZE,IMSIZE,3]) 
x = input_layer

x = Conv2D(64,(7,7),strides=(2,2),padding='same',activation='relu')(x) 
x = BatchNormalization(axis=3)(x) 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x) 

x = Conv2D(192,(3,3),strides=(1,1),padding='same',activation='relu')(x) 
x = BatchNormalization(axis=3)(x) 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x) 

for i in range(9):
     #第三层:Inception 3a。后面的3b，4a-4e，5a，5b都用3a近似替代
     ##64个1x1的卷积核
     branch1x1 = Conv2D(64,(1,1),strides=(1,1),padding='same',activation='relu')(x)
     branch1x1 = BatchNormalization(axis=3)(branch1x1)
     ##96个1x1的卷积核，再进行128个3x3的卷积
     branch3x3 = Conv2D(96,(1,1),strides=(1,1),padding='same',activation='relu')(x) 
     branch3x3 = BatchNormalization(axis=3)(branch3x3) 
     branch3x3 = Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu')(branch3x3) 
     branch3x3 = BatchNormalization(axis=3)(branch3x3)
     ##16个1x1的卷积核，再进行32个5x5的卷积
     branch5x5 = Conv2D(16,(1,1),strides=(1,1),padding='same',activation='relu')(x) 
     branch5x5 = BatchNormalization(axis=3)(branch5x5)
     branch5x5 = Conv2D(32,(5,5),strides=(1,1),padding='same',activation='relu')(branch5x5) 
     branch5x5 = BatchNormalization(axis=3)(branch5x5) 
     ##pool层，使用3x3的核，输出28x28x192，然后进行32个1x1的卷积
     branchpool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
     branchpool = Conv2D(32,(1,1),strides=(1,1),padding='same',activation='relu')(branchpool) 
     branchpool = BatchNormalization(axis=3)(branchpool) 
     x = concatenate([branch1x1,branch3x3,branch5x5,branchpool],axis=3)
     x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

x = Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(17,activation='softmax')(x)
output_layer=x
model=Model(input_layer,output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________